In [ ]:
# hyundaicar_descript_merge_all.json


In [7]:
import json
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from pathlib import Path

# 1. 데이터 로딩
with open('result_dataset/hyundaicar_descript_merge_all.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

descriptions = [item["description"] for item in data]
car_info = [
    {
        "car_name": item["car_name"],
        "category": item["category"],
        "image_path": item["image_path"]
    } for item in data
]

# 2. 텍스트 임베딩
model = SentenceTransformer('jhgan/ko-sroberta-multitask')
embeddings = model.encode(descriptions, show_progress_bar=True)

# 3. Qdrant 클라이언트 연결 및 컬렉션 생성 (로컬 파일 DB 모드)
QDRANT_PATH = Path("qdrant")  # 원하는 경로로 변경 가능
COLLECTION_NAME = "hyundaicar_embeddings"
EMBEDDING_DIM = embeddings[0].shape[0] if hasattr(embeddings[0], "shape") else len(embeddings[0])

client = QdrantClient(
    path=QDRANT_PATH,
    prefer_grpc=False  # 로컬 파일 DB 모드 필수
)

client.recreate_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(
        size=EMBEDDING_DIM,
        distance=Distance.COSINE
    )
)
print("✅ Qdrant 컬렉션 생성 완료")

# 4. 데이터 업로드
points = [
    PointStruct(
        id=i,
        vector=embeddings[i],
        payload=car_info[i]
    )
    for i in range(len(embeddings))
]

client.upsert(collection_name=COLLECTION_NAME, points=points)

print("✅ Qdrant에 벡터 및 metadata 업로드 완료!")

Batches: 100%|██████████| 2/2 [00:01<00:00,  1.41it/s]
/var/folders/69/bt3_5fns57n_cp5_ydbrhfh80000gn/T/ipykernel_93834/4042269537.py:34: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


✅ Qdrant 컬렉션 생성 완료
✅ Qdrant에 벡터 및 metadata 업로드 완료!


In [12]:
type(client)

qdrant_client.qdrant_client.QdrantClient

In [13]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from tools import TOOLS, image_node, generate_node

# 환경변수 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4.o-mini")

def check_route(state):
    user_input = state["input"]
    if "이미지" in user_input or "image" in user_input:
        return "image"
    elif "검색" in user_input or "찾아" in user_input or "RAG" in user_input:
        return "rag"
    else:
        return "llm"

def rag_node(state):
    query = state["input"]
    query_vec = model.encode([query])[0]
    hits = client.search(
        collection_name=COLLECTION_NAME,
        query_vector=query_vec,
        limit=3
    )
    retrieved = [hit.payload for hit in hits]
    state["intermediate"] = f"RAG 검색 결과: {retrieved}"
    return state

def llm_node(state):
    response = llm.invoke(state["input"])
    state["intermediate"] = response.content
    return state

def synthesize_response(state):
    return {"output": state.get("intermediate", "DB에 있는 이미지 또는 일반 응답")}

def start(state):
    if isinstance(state, str):
        return {"input": state}
    return state

# 아래 함수들은 이미 tools.py에 정의되어 있다고 가정
# from tools import image_node, generate_node

checkpointer = MemorySaver()
graph = StateGraph(dict)  # GraphState 대신 dict로도 충분

graph.add_node("start", start)
graph.add_node("check_route", check_route)
graph.add_node("image", image_node)           # tools.py에서 import
graph.add_node("generate", generate_node)     # tools.py에서 import
graph.add_node("rag", rag_node)
graph.add_node("llm", llm_node)
graph.add_node("synthesize_response", synthesize_response)

graph.set_entry_point("start")
graph.add_edge("start", "check_route")

# check_route 분기
graph.add_conditional_edges(
    "check_route",
    check_route,  # 반드시 문자열 반환 함수
    {
        "image": "image",
        "rag": "rag",
        "llm": "llm"
    }
)

# image → generate → synthesize_response
graph.add_edge("image", "generate")
graph.add_edge("generate", "synthesize_response")

# rag/llm은 바로 synthesize_response로
graph.add_edge("rag", "synthesize_response")
graph.add_edge("llm", "synthesize_response")

graph.add_edge("synthesize_response", END)

final_graph = graph.compile(checkpointer=checkpointer)

In [14]:
from IPython.display import Image

img = final_graph.get_graph().draw_mermaid_png()
Image(img)

# result = flow.invoke("안녕")
# print(result)

InvalidUpdateError: At key '__root__': Can receive only one value per step. Use an Annotated key to handle multiple values.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_CONCURRENT_GRAPH_UPDATE